<a href="https://colab.research.google.com/github/nerdk312/Model-based-RL/blob/master/Model_based_Pacman_All_Agents_LSTM_040520.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
'''
function ClickConnect(){
console.log("Working"); 
document.querySelector("colab-toolbar-button#connect").click() 
}
setInterval(ClickConnect,60000)
'''

# Installation and Imports

Used to save Pacman video

In [0]:
#remove " > /dev/null 2>&1" to see what is going on under the hood
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1

In [2]:
!apt-get update > /dev/null 2>&1
!apt-get install cmake > /dev/null 2>&1
!pip install --upgrade setuptools 2>&1
!pip install ez_setup > /dev/null 2>&1
!pip install gym[atari] > /dev/null 2>&1
!apt-get install ffmpeg

Requirement already up-to-date: setuptools in /usr/local/lib/python3.6/dist-packages (46.1.3)
Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.6-0ubuntu0.18.04.1).
0 upgraded, 0 newly installed, 0 to remove and 111 not upgraded.


In [3]:
#!pip install git+git://github.com/mila-iqia/atari-representation-learning.git
!pip install git+git://github.com/ankeshanand/pytorch-a2c-ppo-acktr-gail
!pip install git+git://github.com/openai/baselines
!pip install wandb

  Cloning git://github.com/ankeshanand/pytorch-a2c-ppo-acktr-gail to /tmp/pip-req-build-saj7m472
  Running command git clone -q git://github.com/ankeshanand/pytorch-a2c-ppo-acktr-gail /tmp/pip-req-build-saj7m472
  Created wheel for a2c-ppo-acktr: filename=a2c_ppo_acktr-0.0.1-cp36-none-any.whl size=18833 sha256=dea7266d202cd368e817d8e8181a0939a845c187c44595e8c4050bd04bd5b706
  Stored in directory: /tmp/pip-ephem-wheel-cache-k14hs4ah/wheels/91/52/02/ec5c530fd76d56a66934ee91abbdae5240b766be1dc176deb7
Successfully built a2c-ppo-acktr
  Cloning git://github.com/openai/baselines to /tmp/pip-req-build-nqlk76vx
  Running command git clone -q git://github.com/openai/baselines /tmp/pip-req-build-nqlk76vx
     |████████████████████████████████| 1.6MB 3.1MB/s 
  Created wheel for baselines: filename=baselines-0.1.6-cp36-none-any.whl size=220664 sha256=6e91c0fd0d491893745206b04c630dd7c788322382a9d8befd5007c363ba9f29
  Stored in directory: /tmp/pip-ephem-wheel-cache-avzb72l9/wheels/42/1c/91/28314e0c

In [0]:
import os
from __future__ import print_function
import pickle
import sys
sys.path.append('/content/drive/My Drive/Unsupervised_state_representation/atariari')

import wandb

import random
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import numpy as np

from tqdm import tqdm
import datetime
import time

from benchmark.envs import *
from torch.autograd import Variable

import gym
from benchmark.wrapper import AtariARIWrapper

#from benchmark import *
#from methods import utils

# Needed to create dataloaders
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from custom.EarlyStopping import EarlyStopping_loss
from custom.model import *
from custom.LSTM_model import *
from custom.custom_wrappers import DeterminsticNoopResetEnv 
# Imported required for the Model-based RL
#from sklearn.preprocessing import StandardScaler
from baselines.common.atari_wrappers import EpisodicLifeEnv

In [5]:
!wandb login #######################

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
Successfully logged in to Weights & Biases!


# General Functions

In [0]:
class General_functions():
    def __init__(self, ENV_NAME,n_actions,possible_positions):
        self.ENV_NAME = ENV_NAME
        self.env = gym.make(self.ENV_NAME)
        self.env = EpisodicLifeEnv(DeterminsticNoopResetEnv(self.env,66))        
        self.env = AtariARIWrapper(self.env)
        self.initial_info_labels = self.env.labels()
        self.key_list = ['enemy_blinky_x', 'enemy_blinky_y','enemy_pinky_x','enemy_pinky_y','enemy_inky_x','enemy_inky_y', 'enemy_sue_x', 'enemy_sue_y', 'player_x', 'player_y']
        
    
        self.prev_action_counter =  False
        self.repeated_end = False # Nawid -  Sets self.repeated end as false initially
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.n_actions = n_actions
        # possible positions is numpy array with most of the possible positions that the agent can go to
        self.possible_positions = possible_positions
        self.possible_positions_list = self.possible_positions.tolist()

    def one_hot(self,i):
        a = np.zeros(self.n_actions, 'uint8')
        a[i-1] = 1
        return a

    def state_conversion(self,obs,info_labels):
        state = [info_labels[word] for word in self.key_list if word in info_labels]                
        state = np.array(state)
        return state
    
    def next_position(self,state, action):
        next_position = state[-2:].copy()
        #print(next_position)
        if action == 1:
            next_position[1] = next_position[1] - 2 
        elif action == 2:
            next_position[0] = next_position[0] + 2
        elif action == 3:
            next_position[0] = next_position[0] - 2
        elif action == 4:
            next_position[1] = next_position[1] + 2
        
        if next_position.tolist() in self.possible_positions_list: # possible positions will be a list which is fed into the network 
            return True
        else:
            return False
            
    ''' This is a general case which can be used with both the controller and the data collector
    def random_action_selection(self, state,prev_action = None):
        while True:
            action = np.random.randint(1,5) 
            feasible_action = self.next_position(state,action)
            if feasible_action:
                self.prev_action_counter = True
                #print('action:',action)
                return action
            else:
                if self.prev_action_counter and prev_action !=None:
                    #print('repeated action:',prev_action)
                    return prev_action
    '''

    
    def check_all_agents(self,info_label,next_info_label):
        repeated = np.equal(info_label,next_info_label).all()
        if repeated:
            self.repeated_end= True

    def check_state(self,state, next_state):
        #print(state[2:])
        repeated_state = np.equal(state[-2:], next_state[-2:]).all()
        if repeated_state:
            self.prev_action_counter = False


# Data collection and preprocessing

In [7]:
class Data_collection(General_functions):
    def __init__(self,ENV_NAME,n_actions, possible_positions):
        super(Data_collection,self).__init__(ENV_NAME,n_actions,possible_positions)
        self.dataset_random = [] # Where the data is saved each time

    def collate_data(self,random_dataset, rl_dataset):
        rand_data = np.array(random_dataset)
        num_rand_examples = len(rand_data)
        D_train = rand_data[:int(-num_rand_examples*1/5)] 
        D_valid = rand_data[int(-num_rand_examples*1/5):]
        print("number random examples:",num_rand_examples, 'len(D_train_rand)', len(D_train),'len(D_valid_rand)', len(D_valid))
        if len(rl_dataset) > 0:
            # Adds the rl dataset to the random dataset if there is any present
            rl_data = np.array(rl_dataset)
            num_rl_examples = len(rl_data)
            D_rl_train = rl_data[:int(-num_rl_examples*1/5)] 
            D_rl_valid = rl_data[int(-num_rl_examples*1/5):]
                        
            D_train = np.concatenate([D_train, D_rl_train], axis = 0)
            D_valid = np.concatenate([D_valid, D_rl_valid], axis = 0)
            print("number rl examples:",num_rl_examples, 'len(D_rl_train)', len(D_rl_train),'len(D_valid_rand)', len(D_rl_valid))
            
        #print("len(D_train):", len(D_train), 'len(D_valid)', len(D_valid))

        # Shuffle the dataset
        
        sff = np.arange(len(D_train))
        np.random.shuffle(sff)
        D_train = D_train[sff]
        
        #print('D_train shape',D_train.shape)

        # Create the input and output for the train
        X_train_obs = np.array([obs for obs,_,_,_,_ in D_train]) # Takes obs and action
        X_train_obs = X_train_obs.astype(np.float32) # Need to change it to a int16 so it is signed ( so negative values can be calculated)
        X_train_act = np.array([act for _,_,_,_,act in D_train])
        
        # Env output
        y_env_train = np.array([no for _,no,_,_,_ in D_train])
        #print('y train:',y_env_train[0:100,:])
        #print('X train:',X_train_obs[0:100,:])
        y_env_train = y_env_train.astype(np.float32) # Need to change it to a int16 so it is signed ( so negative values can be calculated)
        

        y_env_train = y_env_train - X_train_obs 
        
        # Next state output
        X_val_obs = np.array([obs for obs,_,_,_,_ in D_valid]) # Takes obs and action
        X_val_obs = X_val_obs.astype(np.float32) # Need to change it to a int16 so it is signed ( so negative values can be calculated)        
        X_val_act = np.array([act for _,_,_,_,act in D_valid])

        y_env_val = np.array([no for _,no,_,_,_ in D_valid])
        y_env_val = y_env_val.astype(np.float32)
        y_env_val = y_env_val - X_val_obs 

        env_train_data, env_val_data = (X_train_obs, X_train_act, y_env_train), (X_val_obs, X_val_act, y_env_val)
        return env_train_data, env_val_data 

    def setup_dataset(self,env_train_data, env_val_data):
        # Unpack data
        (X_env_train_obs, X_env_train_act, y_env_train), (X_env_val_obs, X_env_val_act, y_env_val) = env_train_data, env_val_data
    
        # Concatentates the normalised states with the one hot vector for the actions
        X_env_train = np.concatenate((X_env_train_obs,X_env_train_act),axis=1)
        X_env_val = np.concatenate((X_env_val_obs,X_env_val_act),axis=1)

        # Pack data tuples
        env_train_data, env_val_data = (X_env_train, y_env_train),(X_env_val, y_env_val) 
        return env_train_data, env_val_data
    
    def random_action_selection(self, state,prev_action = None):
        while True:
            action = np.random.randint(1,5) 
            feasible_action = self.next_position(state,action)
            if feasible_action:
                self.prev_action_counter = True
                #print('action:',action)
                return action, None
            else:
                infeasible_action_one_hot = self.one_hot(action)
                #self.dataset_random.append([state, state, 0, False, infeasible_action_one_hot]) # NEED TO CHANGE WHERE THE DATA IS SAVED WHEN I AM USING THE DONES SINCE THE DONES ARE ADDED IN THE MAIN FUNCTION TO PAST VALUES   
                if self.prev_action_counter and prev_action !=None:
                    #print('repeated action:',prev_action)
                    return prev_action, infeasible_action_one_hot
    
    def gather_random_trajectories(self,num_traj):
        #dataset_random = []
        
        for n in range(num_traj):
            if n % 10 ==0:
                print('trajectory number:',n)
                # Initial set up
            #self.env.seed(0)
            self.env = gym.make(self.ENV_NAME)
            self.env = EpisodicLifeEnv(DeterminsticNoopResetEnv(self.env,noop_max=66))
            self.env = AtariARIWrapper(self.env)
            obs = self.env.reset()
            
            self.repeated_end = False
            info_labels = self.env.labels() # Nawid - Used to get the current state
            state = self.state_conversion(obs, info_labels) # Used to get the initial state
            prev_action = None # Initialise prev action has having no action
            
            while True:
                sampled_action, infeasible_action_one_hot = self.random_action_selection(state,prev_action)
                #sampled_action = np.random.randint(1,5)
                 
                sampled_action_one_hot = self.one_hot(sampled_action)
                next_obs, reward, done, next_info = self.env.step(sampled_action)
                next_info_labels = next_info['labels']
                
                next_state = self.state_conversion(next_obs, next_info_labels)
                self.check_state(state,next_state)
                self.check_all_agents(info_labels, next_info_labels) # need to use the info labels to predict the state as the info labels have all the informaiton
                    
                if not self.repeated_end:
                    
                    if infeasible_action_one_hot is not None:
                        #print(infeasible_action_one_hot)
                        fake_next_state = np.zeros_like(state) #  Need to instantiate a new version each time to prevent updating a single variable which will affect all places(eg lists) where the variable is added
                        fake_next_state[0:-2] = next_state[0:-2].copy() # the enemy position of the fake next state is the current enemy position
                        fake_next_state[-2:] = state[-2:].copy() # The agent position for the fake next state is the state before any action was taken

                        self.dataset_random.append([state, fake_next_state, 0, False, infeasible_action_one_hot])

                    self.dataset_random.append([state, next_state, reward, done,sampled_action_one_hot])
                else:
                    self.dataset_random[-1][-2] = True # sets the previous value to done
                    done = True   

                state = next_state.copy()
                info_labels = next_info_labels.copy()
                prev_action = sampled_action
                if done:
                    break    

        return self.dataset_random    

ENV_NAME = 'MsPacman-ramDeterministic-v4'
n_actions = 4 #9 - Nawid - Change to 5 actions as the 4 other actions are simply copies of the other actions, therefore 5 actions should lower the amount of data needed.

possible_positions = np.load('/content/drive/My Drive/MsPacman-data/possible_pacman_positions.npy',allow_pickle=True)
data_object = Data_collection(ENV_NAME,n_actions,possible_positions)
data1 = data_object.gather_random_trajectories(10)

rl_dataset = []
env_train, env_val = data_object.collate_data(data1,rl_dataset)
full_env_train, full_env_val = data_object.setup_dataset(env_train,env_val)


trajectory number: 0
number random examples: 1828 len(D_train_rand) 1463 len(D_valid_rand) 365


# Hyperparameters

In [11]:
# General
n_actions = 4
# Time and date information
now = datetime.datetime.now()
date_time = "{}_{}_{}-{}_{}".format(now.day,now.month,now.year, now.hour, now.minute)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
# Data related
possible_positions = np.load('/content/drive/My Drive/MsPacman-data/possible_pacman_positions.npy',allow_pickle=True)
no_agents = 5 # number of agents present

# Making the network deterministic - https://pytorch.org/docs/stable/notes/randomness.html
random_seed = 0
if random_seed:    
    torch.manual_seed(1)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    #np.random.seed(0)

compulsory_dict = {'ENV_NAME': 'MsPacman-ramDeterministic-v4',
                   'num_rand_trajectories': 3000,
                   'device':'cuda' if torch.cuda.is_available() else 'cpu',
                   'pretrain_model': False,
                   'num_RNN_layers': 1,
                   'num_directions': 1,
                   'input_dim': (no_agents*2) + n_actions,
                   'timesteps': 5,
                   'BATCH_SIZE':128,
                   'hidden_dim':1024,
                   'output_dim': no_agents*2,
                   'lr': 1e-4,
                   'clip_grad':0.1,
                   'AGGR_ITER':10,
                   'load_data': True,
                   'collect_data':False,
                   'TRAIN_MODEL_ITER':1000,
                   'STEPS_PER_AGGR':1000,
                   'HORIZON_LENGTH':1,
                   'NUM_ACTIONS_SEQUENCES':1,
                   'random_seed':random_seed}

wandb.init(entity="nerdk312", project="ATARI_LABELS_MPC",config = compulsory_dict)

config = wandb.config
if compulsory_dict['load_data']:
    loaded_trajectories = '/content/drive/My Drive/MsPacman-data/rand_traj_3000-4_5_2020-10_44.npy' #'/content/drive/My Drive/MsPacman-data/rand_traj_3000-3_5_2020-11_23.npy'
    config.loaded_trajectories = loaded_trajectories

if compulsory_dict['pretrain_model']:
    pretrained_model_dir = '/content/drive/My Drive/MsPacman-data/Env model/Env_model_21_4_2020-8_37.pt'
    config.pretrained_model_dir = pretrained_model_dir

cuda


# Main - Model training

Data collection and processing

In [9]:
# gather the dataset of random sequences
data_collector = Data_collection(compulsory_dict['ENV_NAME'],n_actions,possible_positions)
if compulsory_dict['load_data']:
    rand_dataset = np.load(loaded_trajectories,allow_pickle=True)
else:
    rand_dataset = data_collector.gather_random_trajectories(compulsory_dict['num_rand_trajectories'])
    filename = '/content/drive/My Drive/MsPacman-data/rand_traj_{}-'.format(compulsory_dict['num_rand_trajectories']) + date_time
    np.save(filename,rand_dataset)

empty_dataset = []
env_train_data, env_val_data = data_collector.collate_data(rand_dataset,empty_dataset)
norm_env_train_data, norm_env_val_data = data_collector.setup_dataset(env_train_data, env_val_data)

number random examples: 428516 len(D_train_rand) 342813 len(D_valid_rand) 85703


Dynamics model training

In [12]:
#env_model = NNDynamicsModel(n_actions + 2, 2).to(device) # Nawid - Need to put both actions as it is a one-hot vector
env_model = recurrent_dynamics_model(compulsory_dict['input_dim'], compulsory_dict['hidden_dim'], compulsory_dict['output_dim']).to(device)
if compulsory_dict['pretrain_model']:
    #env_model.load_state_dict(torch.load(pretrained_env_model))
    checkpoint = torch.load(pretrained_env_model)
    env_model.load_state_dict(checkpoint['model_state_dict'])
    
    for parameter in env_model.parameters():
        parameter.requires_grad = False
    env_model.eval()
else:
    env_optimizer = torch.optim.Adam(env_model.parameters(),compulsory_dict['lr']) # Nawid - Optimizer for the env model
    wandb.watch(env_model, log="all")
    env_model_name = 'Env_model'+ '_' + date_time
    early_stopping_env = EarlyStopping_loss(patience=5, verbose=True, wandb=wandb, name=env_model_name)
    for n_iter in range(compulsory_dict['AGGR_ITER']):
        if early_stopping_env.early_stop:
            print('Early stopping')
            break 
        train_recurrent_model(norm_env_train_data, norm_env_val_data, env_model, compulsory_dict['BATCH_SIZE'],compulsory_dict['timesteps'], compulsory_dict['TRAIN_MODEL_ITER'],env_optimizer,device,early_stopping_env)

  0%|          | 0/1000 [00:00<?, ?it/s]

17140
Validation loss decreased/improved for Env_model_4_5_2020-11_20  (100000000000.000000 --> 2.038753).  Saving model ...


  0%|          | 1/1000 [00:08<2:25:25,  8.73s/it]

epoch: 0 completed
17140
Validation loss decreased/improved for Env_model_4_5_2020-11_20  (2.038753 --> 1.505102).  Saving model ...


  0%|          | 2/1000 [00:17<2:22:56,  8.59s/it]

...

EarlyStopping for Env_model_4_5_2020-11_20 counter: 2 out of 5
epoch: 74 completed


  8%|▊         | 76/1000 [12:03<2:15:04,  8.77s/it]

17140
EarlyStopping for Env_model_4_5_2020-11_20 counter: 3 out of 5
epoch: 75 completed


  8%|▊         | 77/1000 [12:11<2:11:21,  8.54s/it]

17140
EarlyStopping for Env_model_4_5_2020-11_20 counter: 4 out of 5
epoch: 76 completed
17140
EarlyStopping for Env_model_4_5_2020-11_20 counter: 5 out of 5
Env_model_4_5_2020-11_20 has stopped
Early stopping
Early stopping


# Prediction testing

In [0]:
x_val, y_val = norm_env_val_data

batch_size = 2
timesteps = 5
hidden_dim = 1024
num_layers = 1
num_directions = 1
test_inputs = torch.tensor(x_val[0:timesteps*batch_size,:])
test_inputs = test_inputs.view(timesteps,batch_size,-1).to(device)
print(test_inputs.size())

targets = torch.tensor(y_val[0:timesteps*batch_size,:])
targets = targets.view(timesteps,batch_size,-1).to(device)
print(targets.size())
    
new_states = (torch.zeros(num_layers*num_directions,batch_size,hidden_dim).to(device),
          torch.zeros(num_directions*num_layers,batch_size,hidden_dim).to(device))

outputs,_ = env_model(test_inputs, new_states)
print('outputs:',outputs)
print('targets:',targets)

Training the reward function and MPC

# Old Code

In [0]:
loaded_trajectories = '/content/gdrive/My Drive/MsPacman-data/rand_traj_3000-29_4_2020-9_8.npy'
loaded_dataset = np.load(loaded_trajectories,allow_pickle=True)

loaded_trajectory_2 = '/content/gdrive/My Drive/MsPacman-data/rand_traj_3000-29_4_2020-9_50.npy'
loaded_dataset2 = np.load(loaded_trajectory_2,allow_pickle=True)

collated_dataset = np.concatenate((loaded_dataset,loaded_dataset2),axis=0)
filename = '/content/gdrive/My Drive/MsPacman-data/only_agent_position_6000_traj__prev_action_fix+failed_action_fix'
np.save(filename,collated_dataset)

In [0]:
def state_conversion(self,obs,info_labels):
        state = []          
        for key in info_labels:
            if key in self.key_list: # Nawid - Only the first 14 info are crucial I believe
                    state.append(info_labels[key])
                    
        state = np.array(state)
        return state


ENV_NAME = 'MsPacman-ramDeterministic-v4'
env = gym.make(ENV_NAME)
env = AtariARIWrapper(env)
obs = env.reset()
info_labels = env.labels()
print(info_labels)
state = []
key_list = ['enemy_blinky_x', 'enemy_blinky_y','enemy_pinky_x','enemy_pinky_y', 'player_x', 'player_y']
for key in info_labels:
    if key in key_list:
        state.append(info_labels[key])
        print(key)

state = [info_labels[word] for word in key_list if word in info_labels ]
state = np.array(state)
print(state.shape)
for word in key_list:
    if word in info_labels:
        print(word)
        print(info_labels[word])
print(state)
print('agent state',state[:-3])
'''
new_l = []
for i in l:
    if i['term'] in words:
        new_l.append(i)
#print([i for i in info_labels if i["term"] in info_labels])
'''